# --- Day 13: Shuttle Search ---

https://adventofcode.com/2020/day/13

In [1]:
path = '../inputs/'

## Part 1

In [42]:
def part(filename):
    with open (path + filename) as file:
        lines = file.readlines() 
        print(lines)
    arrival_time = int(lines[0].strip())
    bus_schedule = lines[1].strip()
    buses = [int(n) for n in bus_schedule.split(',') if not n == 'x']
    print(arrival_time)
    print(bus_schedule)
    print(buses)
    
    wait_times = [b - (arrival_time % b)  for b in buses]
    print(wait_times)
    
    min_wait_time = min(wait_times)
    print(min_wait_time)
    bus_with_min_wait_time = buses[wait_times.index(min_wait_time)]
    print(bus_with_min_wait_time)
    
    return min_wait_time * bus_with_min_wait_time

In [43]:
foo('example_bus_notes.txt') # Should return 295

['939\n', '7,13,x,x,59,x,31,19\n']
939
7,13,x,x,59,x,31,19
[7, 13, 59, 31, 19]
[6, 10, 5, 22, 11]
5
59


295

In [ ]:
def some_func():
    
    yield 

In [25]:
939 / 7

134.14285714285714

In [26]:
7 * 134

938

## Part 2